In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### Specify the headers of the dataframes and load the csv into tow seperate data frames


In [2]:
# Correcting the headers lists: each column name should be a separate string within the list.
df1_headers = [
    "Station_ID", "Date_Time", "altimeter_set_1", "air_temp_set_1", "relative_humidity_set_1",
    "wind_speed_set_1", "wind_direction_set_1", "wind_gust_set_1", "solar_radiation_set_1",
    "precip_accum_24_hour_set_1", "precip_accum_since_local_midnight_set_1",
    "wind_chill_set_1d", "wind_cardinal_direction_set_1d", "heat_index_set_1d",
    "dew_point_temperature_set_1d", "pressure_set_1d", "sea_level_pressure_set_1d"
    # Make sure all headers are included and separated correctly
]

df2_headers = [
    "Station_ID", "Date_Time", "altimeter_set_1", "air_temp_set_1", "dew_point_temperature_set_1",
    "relative_humidity_set_1", "wind_speed_set_1", "wind_direction_set_1", "wind_gust_set_1",
    "sea_level_pressure_set_1", "weather_cond_code_set_1", "cloud_layer_3_code_set_1",
    "pressure_tendency_set_1", "precip_accum_one_hour_set_1", "precip_accum_three_hour_set_1",
    "cloud_layer_1_code_set_1", "cloud_layer_2_code_set_1", "precip_accum_six_hour_set_1",
    "precip_accum_24_hour_set_1", "visibility_set_1", "metar_remark_set_1", "metar_set_1",
    "air_temp_high_6_hour_set_1", "air_temp_low_6_hour_set_1", "peak_wind_speed_set_1",
    "ceiling_set_1", "pressure_change_code_set_1", "air_temp_high_24_hour_set_1",
    "air_temp_low_24_hour_set_1", "peak_wind_direction_set_1", "wind_chill_set_1d",
    "wind_cardinal_direction_set_1d", "heat_index_set_1d", "weather_condition_set_1d",
    "weather_summary_set_1d", "cloud_layer_1_set_1d", "cloud_layer_2_set_1d",
    "cloud_layer_3_set_1d", "dew_point_temperature_set_1d", "pressure_set_1d",
    "sea_level_pressure_set_1d"
    # Again, ensure all headers are separated correctly
]

# Read the CSV files with the corrected header alignment
df1 = pd.read_csv("Data/G3425.csv", names=df1_headers, skiprows=8, index_col=False)
df2 = pd.read_csv("Data/KHYI.csv", names=df2_headers, skiprows=8, index_col=False)
# Inspect the first few rows of the 'Date_Time' column
print(df1['Date_Time'].head())
print(df2['Date_Time'].head())

import pandas as pd

# Define a function to remove timezone information
def remove_timezone(dt_str):
    return dt_str[:-4]  # Adjust slicing based on your data format

# Apply this function to your 'Date_Time' columns
df1['Date_Time'] = df1['Date_Time'].apply(remove_timezone)
df2['Date_Time'] = df2['Date_Time'].apply(remove_timezone)

# Specify the format of your date-time strings
date_format = "%m/%d/%Y %H:%M"  # Adjust this format to match your data

# Convert 'Date_Time' to datetime
df1['Date_Time'] = pd.to_datetime(df1['Date_Time'], format=date_format, errors='coerce')
df2['Date_Time'] = pd.to_datetime(df2['Date_Time'], format=date_format, errors='coerce')

# Optional: Localize to a specific timezone if needed
# df1['Date_Time'] = df1['Date_Time'].dt.tz_localize('America/Chicago')
# df2['Date_Time'] = df2['Date_Time'].dt.tz_localize('UTC')

# Rename columns
df1.rename(columns={'Date_Time': 'timestamp'}, inplace=True)
df2.rename(columns={'Date_Time': 'timestamp'}, inplace=True)
df3 = pd.read_csv("Data/Meadow Center Sensor Data Test.csv")
df3 = df3.drop(columns=['Month','Day','Year','Date'])
df3.rename(columns={'Taken At': 'timestamp'}, inplace=True)
df4 = pd.read_csv('Data/usgs.waterservices.csv',skiprows=1)
df4.rename(columns={'20d': 'timestamp'}, inplace=True)
date_range = pd.date_range(start= '2022-06-11', end = '2023-06-11', freq='15S')
df3.rename(columns={'Temperature': 'Water Temperature'}, inplace=True)

final_df = pd.DataFrame(date_range, columns=['timestamp'])

0    05/03/2023 13:37 CDT
1    05/03/2023 13:47 CDT
2    05/03/2023 14:07 CDT
3    05/03/2023 14:47 CDT
4    05/03/2023 15:07 CDT
Name: Date_Time, dtype: object
0    06/11/2022 23:00 UTC
1    06/11/2022 23:05 UTC
2    06/11/2022 23:10 UTC
3    06/11/2022 23:15 UTC
4    06/11/2022 23:20 UTC
Name: Date_Time, dtype: object


/tmp/ipykernel_9948/3643472824.py:64: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  date_range = pd.date_range(start= '2022-06-11', end = '2023-06-11', freq='15S')


### This function is used to reset the timezones in the dataset because the usgs and Sensor Data use UTC time zone

In [3]:
import pytz
dfs = [df1, df2, df3, df4]
# Function to parse datetime with different formats and timezones
def parse_datetime(dt):
    try:
        # Try parsing as is (if no timezone info, etc.)
        return pd.to_datetime(dt)
    except ValueError:
        # Handle entries with 'CDT' and 'UTC' separately
        if 'CDT' in dt:
            dt = dt.replace(' CDT', '')  # Remove 'CDT'
            parsed_dt = pd.to_datetime(dt, format='%m/%d/%Y %H:%M')  # Parse the datetime
            central = pytz.timezone('America/Chicago')
            return parsed_dt.tz_localize(central).tz_convert(pytz.utc).tz_localize(None)  # Convert to UTC and remove tz info
        elif 'UTC' in dt:
            dt = dt.replace(' UTC', '')  # Remove 'UTC'
            return pd.to_datetime(dt, format='%m/%d/%Y %H:%M')  # Parse the datetime
        else:
            # Custom parsing for other formats can be added here
            return pd.to_datetime(dt)  # Or a default return, if it's a format pandas can parse by default

# Iterate over all DataFrames
for dataframe in dfs:
    # Apply the conversion function to the 'Date_Time' column
    dataframe['timestamp'] = dataframe['timestamp'].apply(parse_datetime)

    # If you want to rename 'Date_Time' to 'timestamp', uncomment the following line
    # dataframe.rename(columns={'Date_Time': 'timestamp'}, inplace=True)

# Now, all your 'Date_Time' columns should have a unified format, and you can proceed with combining your DataFrames
# Iterate over all DataFrames
for dataframe in dfs:
    # Apply the conversion function to the 'Date_Time' column
    dataframe['timestamp'] = dataframe['timestamp'].apply(parse_datetime)

    # If you want to rename 'Date_Time' to 'timestamp', uncomment the following line
    #dataframe.rename(columns={'Date_Time': 'timestamp'}, inplace=True)

# Now, all your 'Date_Time' columns should have a unified format, and you can proceed with combining your DataFrames

In [4]:
df1['timestamp'] = df1['timestamp'].dt.tz_localize(None)
df2['timestamp'] = df2['timestamp'].dt.tz_localize(None)
df3['timestamp'] = df3['timestamp'].dt.tz_localize(None)
df4['timestamp'] = df4['timestamp'].dt.tz_localize(None)
df1.set_index('timestamp', inplace=True)
df2.set_index('timestamp', inplace=True)
df3.set_index('timestamp', inplace=True)
df4.set_index('timestamp', inplace=True)
dfs = [df1,df2]
merged_df = pd.concat(dfs, axis=0)
merged_df = pd.merge(merged_df, df3, on='timestamp', how='outer')
merged_df = pd.merge(merged_df, df4, on='timestamp', how='outer')
merged_df.reset_index(inplace=True)
final_df = merged_df
final_df = merged_df.drop(columns=['5s','15s','6s','10s'])

### This cell adds a feature column to the dataframe by dividing the lake into 2 sections, upstream and down stream. The dividing line is around Deep Hole and the Weather Station onsite.

In [5]:
final_df.set_index('timestamp', inplace=True)
final_df = final_df[~final_df.index.duplicated(keep='last')]
all_timestamps = pd.date_range(start='2022-06-11 00:00:00', end='2023-06-11 00:00:00', freq='15S')
final_df = final_df.reindex(all_timestamps, fill_value=pd.NA)
final_df.rename(columns={'index': 'timestamp'}, inplace=True)
final_df = final_df.rename(columns={'14n': 'Discharge Rate'})

/tmp/ipykernel_9948/725395635.py:3: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  all_timestamps = pd.date_range(start='2022-06-11 00:00:00', end='2023-06-11 00:00:00', freq='15S')


In [6]:
#Current shape of Dataset
final_df.shape

(2102401, 47)

# Data Preprocessing
This cell is the first part for our linear interpolation. We take the mean of each column and have it as the first and last entry in each column

In [7]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame 'final_df'

# Iterating over each column in the DataFrame
for col_name in final_df.columns:
    # We're only interested in columns with numeric data
    if pd.api.types.is_numeric_dtype(final_df[col_name]):
        col_mean = final_df[col_name].mean()

        # If col_mean is not NaN, this means that there's at least one non-NaN value in the column
        if not np.isnan(col_mean):
            # Finding the first and last NaN indices in the column
            # We're specifically looking for NaN entries, not just any entry
            first_nan_index = final_df[col_name].index[final_df[col_name].isna()].min()
            last_nan_index = final_df[col_name].index[final_df[col_name].isna()].max()

            # Filling these specific NaN positions with the column mean, if they exist
            if first_nan_index is not np.nan:
                final_df.at[first_nan_index, col_name] = col_mean
            if last_nan_index is not np.nan:
                final_df.at[last_nan_index, col_name] = col_mean

# You can now check your DataFrame to see if the first and last NaNs were replaced appropriately.
final_df.head()

,Station_ID,altimeter_set_1,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,solar_radiation_set_1,precip_accum_24_hour_set_1,precip_accum_since_local_midnight_set_1,...,weather_condition_set_1d,weather_summary_set_1d,cloud_layer_1_set_1d,cloud_layer_2_set_1d,cloud_layer_3_set_1d,Lat,Long,TDS,Water Temperature,Discharge Rate
2022-06-11 00:00:00,NaN,30.012548,73.204991,68.73463,8.306313,153.281394,12.077422,210.451643,0.111647,0.078487,...,NaN,NaN,NaN,NaN,NaN,29.893364,-97.930973,426.075866,71.74583,116.000000
2022-06-11 00:00:15,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.248803
2022-06-11 00:00:30,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-11 00:00:45,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-11 00:01:00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### This cell does forward linear interpolation for the dataset on numeric columns

In [8]:
# Linear interpolation for the numeric columns.
final_df.interpolate(method='linear', limit_direction='forward', inplace=True)

/tmp/ipykernel_9948/2070673214.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  final_df.interpolate(method='linear', limit_direction='forward', inplace=True)


#### This cell is for the non-numeric columns, we do forward and backward filling for these columns

In [9]:
non_numeric_columns = final_df.select_dtypes(exclude='number').columns
final_df[non_numeric_columns] = final_df[non_numeric_columns].fillna(method='ffill')
final_df[non_numeric_columns] = final_df[non_numeric_columns].fillna(method='bfill')

/tmp/ipykernel_9948/2580860926.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df[non_numeric_columns] = final_df[non_numeric_columns].fillna(method='ffill')
/tmp/ipykernel_9948/2580860926.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df[non_numeric_columns] = final_df[non_numeric_columns].fillna(method='bfill')


In [10]:
final_df.head()

,Station_ID,altimeter_set_1,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,solar_radiation_set_1,precip_accum_24_hour_set_1,precip_accum_since_local_midnight_set_1,...,weather_condition_set_1d,weather_summary_set_1d,cloud_layer_1_set_1d,cloud_layer_2_set_1d,cloud_layer_3_set_1d,Lat,Long,TDS,Water Temperature,Discharge Rate
2022-06-11 00:00:00,KHYI,30.012548,73.204991,68.734630,8.306313,153.281394,12.077422,210.451643,0.111647,0.078487,...,haze,clear,NaN,NaN,NaN,29.893364,-97.930973,426.075866,71.745830,116.000000
2022-06-11 00:00:15,KHYI,30.012504,73.209917,68.728196,8.306894,153.282611,12.078243,210.451590,0.111668,0.078487,...,haze,clear,NaN,NaN,NaN,29.893364,-97.930973,426.073231,71.745853,92.248803
2022-06-11 00:00:30,KHYI,30.012460,73.214844,68.721762,8.307474,153.283828,12.079063,210.451537,0.111690,0.078487,...,haze,clear,NaN,NaN,NaN,29.893364,-97.930973,426.070596,71.745875,92.634416
2022-06-11 00:00:45,KHYI,30.012416,73.219771,68.715328,8.308055,153.285045,12.079884,210.451484,0.111712,0.078487,...,haze,clear,NaN,NaN,NaN,29.893364,-97.930973,426.067961,71.745898,93.020030
2022-06-11 00:01:00,KHYI,30.012373,73.224697,68.708894,8.308635,153.286262,12.080704,210.451431,0.111734,0.078487,...,haze,clear,NaN,NaN,NaN,29.893364,-97.930973,426.065326,71.745921,93.405643


In [11]:
#Do Label Encoding for all non-numeric columns
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Iterate over all non-numeric columns
for column in final_df.select_dtypes(exclude='number').columns:
    # Fit and transform the column
    final_df[column] = label_encoder.fit_transform(final_df[column])

final_df.head()

,Station_ID,altimeter_set_1,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,solar_radiation_set_1,precip_accum_24_hour_set_1,precip_accum_since_local_midnight_set_1,...,weather_condition_set_1d,weather_summary_set_1d,cloud_layer_1_set_1d,cloud_layer_2_set_1d,cloud_layer_3_set_1d,Lat,Long,TDS,Water Temperature,Discharge Rate
2022-06-11 00:00:00,1,30.012548,73.204991,68.734630,8.306313,153.281394,12.077422,210.451643,0.111647,0.078487,...,3,1,NaN,NaN,NaN,29.893364,-97.930973,426.075866,71.745830,116.000000
2022-06-11 00:00:15,1,30.012504,73.209917,68.728196,8.306894,153.282611,12.078243,210.451590,0.111668,0.078487,...,3,1,NaN,NaN,NaN,29.893364,-97.930973,426.073231,71.745853,92.248803
2022-06-11 00:00:30,1,30.012460,73.214844,68.721762,8.307474,153.283828,12.079063,210.451537,0.111690,0.078487,...,3,1,NaN,NaN,NaN,29.893364,-97.930973,426.070596,71.745875,92.634416
2022-06-11 00:00:45,1,30.012416,73.219771,68.715328,8.308055,153.285045,12.079884,210.451484,0.111712,0.078487,...,3,1,NaN,NaN,NaN,29.893364,-97.930973,426.067961,71.745898,93.020030
2022-06-11 00:01:00,1,30.012373,73.224697,68.708894,8.308635,153.286262,12.080704,210.451431,0.111734,0.078487,...,3,1,NaN,NaN,NaN,29.893364,-97.930973,426.065326,71.745921,93.405643


In [12]:
df = final_df
# Step 1: Define the mapping of columns to JSON headers
column_mapping = {
    'Station_ID': 'stationID',
    'altimeter_set_1': 'pressure',
    'air_temp_set_1': 'temp',
    'relative_humidity_set_1': 'humidity',
    'wind_speed_set_1': 'windSpeed',
    'wind_direction_set_1': 'winddir',
    'wind_gust_set_1': 'windGust',
    'solar_radiation_set_1': 'solarRadiation',
    'precip_accum_24_hour_set_1': 'precipTotal',
    'precip_accum_since_local_midnight_set_1': 'precipTotal',
    'wind_chill_set_1d': 'windChill',
    'heat_index_set_1d': 'heatIndex',
    'dew_point_temperature_set_1d': 'dewpt',
    'Lat': 'lat',
    'Long': 'lon',
    'Water Temperature': 'Water Temperature',  # Keeping as is
    'TDS': 'TDS'  # Keeping as is
}

# Step 2: Drop columns with None mapping except 'Water Temperature' and 'TDS'
columns_to_keep = list(column_mapping.keys())
df = df[columns_to_keep]

# Step 3: Drop the 'Station_ID' column
df = df.drop(columns=['Station_ID'], errors='ignore')

# Step 4: Rename the columns to match the JSON headers
df = df.rename(columns=column_mapping)
final_df = df

In [13]:

# Display the updated DataFrame
final_df.head()

,pressure,temp,humidity,windSpeed,winddir,windGust,solarRadiation,precipTotal,precipTotal,windChill,heatIndex,dewpt,lat,lon,Water Temperature,TDS
2022-06-11 00:00:00,30.012548,73.204991,68.734630,8.306313,153.281394,12.077422,210.451643,0.111647,0.078487,-4.08,92.179728,60.157697,29.893364,-97.930973,71.745830,426.075866
2022-06-11 00:00:15,30.012504,73.209917,68.728196,8.306894,153.282611,12.078243,210.451590,0.111668,0.078487,-4.08,92.182064,60.158854,29.893364,-97.930973,71.745853,426.073231
2022-06-11 00:00:30,30.012460,73.214844,68.721762,8.307474,153.283828,12.079063,210.451537,0.111690,0.078487,-4.08,92.184399,60.160010,29.893364,-97.930973,71.745875,426.070596
2022-06-11 00:00:45,30.012416,73.219771,68.715328,8.308055,153.285045,12.079884,210.451484,0.111712,0.078487,-4.08,92.186734,60.161166,29.893364,-97.930973,71.745898,426.067961
2022-06-11 00:01:00,30.012373,73.224697,68.708894,8.308635,153.286262,12.080704,210.451431,0.111734,0.078487,-4.08,92.189069,60.162322,29.893364,-97.930973,71.745921,426.065326


In [14]:
#check to make sure there are no nan values
final_df.isnull().sum()

pressure             0
temp                 0
humidity             0
windSpeed            0
winddir              0
windGust             0
solarRadiation       0
precipTotal          0
precipTotal          0
windChill            0
heatIndex            0
dewpt                0
lat                  0
lon                  0
Water Temperature    0
TDS                  0
dtype: int64

In [15]:
!pip install scikit-learn tpot

: 

In [16]:
#use TPOT Regressor to predict the 'TDS' column
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = final_df.drop(columns=['TDS'])
y = final_df['TDS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TPOTRegressor
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2, random_state=42)

# Fit the TPOTRegressor
tpot.fit(X_train, y_train)


Optimization Progress:  35%|███▌      | 42/120 [1:28:44<1:48:39, 83.58s/pipeline] 

In [ ]:
# Evaluate the TPOT model
print(tpot.score(X_test, y_test))


In [ ]:

#print out R^2 score, RMSE, MAE, MSE
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error

# Make predictions
y_pred = tpot.predict(X_test)

# Calculate R^2 score
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2}")

# Calculate RMSE
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")
# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")
# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

In [ ]:
#To get an accuracy score for regression model, set a scorcing parameter where for every row in the dataset, the model will predict the TDS value and compare it to the actual value, if it is within a certain range of RMSE or MSE, it is considered correct
# Calculate the accuracy score
accuracy_score = (1 - (rmse / y_test.mean())) * 100
print(f"Accuracy Score: {accuracy_score:.2f}%")